In [1]:
import random
import numpy as np
import pandas as pd
import re

def export_dataset():
    #Lista de links (arquivos estão dentro do Google Sheets)
    #Foi necessário criar vários arquivos arquivos por conta do tamanho dos arquivos e o google sheets estava apresentando problemas de memória
    test_links = {
        'link_01': 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQvClyfCtk8JD82zXYSipnN7iCJ0XG0V-z6tcII_14W7tQYPpLLOK45kKWLQf8TBfyRNhH9mlEXiowR/pub?output=csv',
        'link_02': 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQq0fLn7Dkg0-Lg2i9FUD0iL-xLVHn1Gtorx8wLlA7flMftChIJf_PXgziaHshCnxlX52bZJZ2EKFsu/pub?output=csv'
    }
    train_links ={
        'link_01':'https://docs.google.com/spreadsheets/d/e/2PACX-1vS2E4FyELPrgf1rPIeyh4lzQTOtftYYK6vyr7m5TN7cYDZTo4tLy1jZQ06mnnwo0NrJ5ZR5IOm2-ndN/pub?output=csv',
        'link_02':'https://docs.google.com/spreadsheets/d/e/2PACX-1vT-NHTDb71tp400NvyVObenotd8uUATgPgl4nlGwvqpwTTMoCmc8bH7YmyvmzBg_TcIAxoNkC0upfoe/pub?output=csv',       
        'link_03':'https://docs.google.com/spreadsheets/d/e/2PACX-1vQux2yo8RJg5YdLAF7t7iOv1e22RLqp7INPPrQkqDdJVzkbkdJl2QQGiW_KWtnw8mxhbxXu6wgV9N-2/pub?output=csv',
        'link_04':'https://docs.google.com/spreadsheets/d/e/2PACX-1vTFoYy6F6IuVXBhhGazTbuu5LUqItSrOYh88SPj41JjAtKckTyrtygOH7ePKIKJfN80o_7sNPip5UaG/pub?output=csv',
        'link_05':'https://docs.google.com/spreadsheets/d/e/2PACX-1vTooCcyaoXnkazJ6HI95oc-Ll45cMY4qnjh6iIa9HVJfhTej2z9EBDVVHgplEhqLfegmeucud1mXMhS/pub?output=csv'
    }
   
    #Download e leitura do arquivo de treinamento
    temp_df_test = pd.DataFrame()
    for i in test_links:
        file_temp = pd.read_csv(test_links[i])
        temp_df_test = pd.concat([temp_df_test,file_temp]) 
    temp_df_test.reset_index(drop=True, inplace=True)
    temp_df_test.drop('Unnamed: 0',axis=1, inplace= True)

    #Download e leitura do arquivo de teste
    temp_df_train = pd.DataFrame()
    for i in train_links:
        file_temp = pd.read_csv(train_links[i])
        temp_df_train = pd.concat([temp_df_train,file_temp]) 
    temp_df_train.reset_index(drop=True, inplace=True)
    temp_df_train.drop('Unnamed: 0',axis=1, inplace= True)    
    
    return temp_df_test, temp_df_train

test, train = export_dataset()

| Nome da coluna | Descrição |
| :----: | :----: |
| test_id | Numero identificador dos itens da lista |
| name | Titulo da lista*|
| item_condition_id | Numero da condição do item, varia de 1 a 5, sendo 1 a pior e 5 a melhor |
| category_name | Categorias da lista |
| brand_name | Nome da marca do item (existe nulos) |
| price | Preço do produto |
| shipping | Taxa de envio, pago por: 1 - pelo vendedor 0 - pelo comprador |
| item_description | Descrição completa do item* |
| date | Data |
| stock | Estoque |

*¹- Nessas colunas existe um valor chamado [rm], esse valor significa "removed" ou removido, significa que o preço (exemplo: $20) foi removido.

In [2]:
#o rm também foi adicionado em descrições completas, visualizar arquivo fixado no discord

In [3]:
#existem valores repitidos na coluna brand_name (exemplo: adidas e adidas NEO, as duas são a mesma coisa)
#necessário tratar esses dados

In [4]:
train_vis = train[0:1000]
test_vis = test[0:1000]

In [5]:
test1 = train.query("name == '[rm]'")

In [6]:
test2 = train.query("item_description == '[rm]'")

In [7]:
train['item_condition_id'].value_counts()

1    640549
3    432161
2    375479
4     31962
5      2384
Name: item_condition_id, dtype: int64

In [8]:
#category_name
#lista de categorias do nome da lista
#divido por caracteres especiais
#necessário limpeza para avaliação, talvez dividir em lista ou dummmies?

In [9]:
train.shape

(1482535, 10)

In [10]:
test_array = train['brand_name'].drop_duplicates().unique()
train['brand_name'].unique().shape

(4810,)

In [11]:
train.info()    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1482535 entries, 0 to 1482534
Data columns (total 10 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   train_id           1482535 non-null  int64 
 1   name               1482535 non-null  object
 2   item_condition_id  1482535 non-null  int64 
 3   category_name      1476208 non-null  object
 4   brand_name         849853 non-null   object
 5   price              1482535 non-null  object
 6   shipping           1482535 non-null  int64 
 7   item_description   1482531 non-null  object
 8   date               1482535 non-null  object
 9   stock              1482535 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 113.1+ MB


In [12]:
# Remove special caracters from a list and return itself
def remove_especial_caracters(dataframe_column):
    complete_array = []
    for n in range(len(dataframe_column)):
        array = str(dataframe_column[n]).split('/')
        for i in range(len(array)):
            x = str(array[i])
            complete_array.append(x)
    return complete_array

train_frame = remove_especial_caracters(train['category_name'])
test_frame = remove_especial_caracters(test['category_name'])

unique, counts = np.unique(test_frame, return_counts=True)
frame = pd.DataFrame(np.stack((unique, counts), axis = 1), columns= ['Category name', 'Count'])

In [13]:
frame['Count'] = pd.to_numeric(frame['Count'])

frame.sort_values(by=['Count'], inplace=True, ascending = False)

In [14]:
frame.loc[frame['Count']>5000]

,Category name,Count
898,Women,319042
80,Beauty,97265
463,Kids,80299
34,Athletic Apparel,62669
721,Shoes,61731
...,...,...
251,Daily & Travel items,5406
575,One-Pieces,5390
684,Rings,5121
731,Skirts,5029


In [15]:
print(frame['Category name'].unique())
print(frame['Category name'].describe())
display (frame[frame['Category name'].isnull()])

['Women' 'Beauty' 'Kids' 'Athletic Apparel' 'Shoes' 'Electronics' 'Makeup'
 'Other' 'Tops & Blouses' 'Men' 'Home' 'Jewelry' 'Pants, Tights, Leggings'
 'Toys' 'Dresses' 'Cell Phones & Accessories' 'Face' 'T-Shirts'
 'Vintage & Collectibles' "Women's Handbags" "Women's Accessories" 'Jeans'
 'Video Games & Consoles' 'Sweaters' 'Underwear' 'Lips' 'Games' 'Handmade'
 'Skin Care' 'Athletic' 'Eyes' 'Sports & Outdoors' 'Home Décor'
 'Coats & Jackets' 'Fragrance' 'Cases, Covers & Skins' 'Shorts'
 'Kitchen & Dining' 'Bras' 'Tops' 'Tank, Cami' 'Swimwear' 'Blouse'
 'Above Knee, Mini' 'Boots' 'Necklaces' 'Makeup Palettes'
 "Men's Accessories" 'Girls 2T-5T' 'Accessories' 'Girls 0-24 Mos'
 'Shirts & Tops' 'Girls (4+)' 'Shoulder Bag' 'Fashion Sneakers' 'T-shirts'
 'Sandals' 'Boys 0-24 Mos' 'Tops & T-Shirts' 'Wallets' 'Boys (4+)'
 'Knee-Length' 'Boys 2T-5T' 'Bracelets' 'Dolls & Accessories'
 'Slim, Skinny' 'Home Décor Accents' 'Tools & Accessories' 'Two-Piece'
 'Action Figures & Statues' 'Hats' 'Media'

,Category name,Count
